# SQLite with Python - Quick Recap

This is a reminder of working with SQL databases using Python's built-in `sqlite3` library. No classes, no ORM - just simple, direct database operations that are quick and efficient for basic tasks.

## Setting up the Database Path with Pathlib

First, let's understand the file structure and why we use `Path.cwd().parent.parent`:

```
Advanced_Programming/
├── code-examples/
│   ├── 04-class_for_sql/
│   │   └── departement_example.ipynb  ← We are here
│   └── 04-data_structures.ipynb
├── exercices/
│   ├── 01-architecture/
│   ├── 02-shell/
│   └── ...
├── lessons/
│   ├── 01-shell-intro.md
│   └── ...
└── data/
    └── villes_france.db  ← We want to reach this
```

```python
from pathlib import Path

p_departement = Path.cwd().parent.parent / "data" / "villes_france.db" 
p_departement
```

**Path breakdown:**
- `Path.cwd()` → Current working directory: `Advanced_Programming/code-examples/04-class_for_sql/`
- `.parent` → Go up one level: `Advanced_Programming/code-examples/`
- `.parent` → Go up another level: `Advanced_Programming/`
- `/ "data" / "villes_france.db"` → Navigate to: `Advanced_Programming/data/villes_france.db`

**Why Pathlib?** It's more readable than string concatenation and works across operating systems (Windows uses `\`, Unix uses `/`).






In [22]:
from pathlib import Path

p_departement = Path.cwd().parent.parent / "data" / "villes_france.db" 
p_departement

PosixPath('/home/kevin-desktop/Documents/Seafile/COURS/Advanced_Programming/data/villes_france.db')

In [23]:
# Check if file exists
if p_departement.exists():
    print("Database found!")
else:
    print(f"Database not found at: {p_departement.resolve()}")

Database found!


In [24]:
import sqlite3
conn = sqlite3.connect(p_departement)
cursor = conn.cursor()
print(f"Connected to database: {p_departement}")

Connected to database: /home/kevin-desktop/Documents/Seafile/COURS/Advanced_Programming/data/villes_france.db


**Key concepts:**
- **Connection (`conn`)**: Represents the database connection
- **Cursor (`cursor`)**: Used to execute SQL commands and fetch results
- Always close connections when done: `conn.close()`

In [16]:
# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    print(f"  • {table[0]}")

  • villes
  • sqlite_sequence
  • departement


**What's happening:**
- `sqlite_master` is SQLite's system table containing metadata
- `WHERE type='table'` filters to show only tables (not indexes, views, etc.)
- `fetchall()` returns all results as a list of tuples

In [ ]:
cursor.execute("select * FROM departement LIMIT 5;")
# Ugly list of dict, no column names
ans = cursor.fetchall()
ans

[(1, '01', 'Ain', 'AIN', 'ain', 'A500'),
 (2, '02', 'Aisne', 'AISNE', 'aisne', 'A250'),
 (3, '03', 'Allier', 'ALLIER', 'allier', 'A460'),
 (4,
  '04',
  'Alpes-de-Haute-Provence',
  'ALPES-DE-HAUTE-PROVENCE',
  'alpes-de-haute-provence',
  'A412316152'),
 (5, '05', 'Hautes-Alpes', 'HAUTES-ALPES', 'hautes-alpes', 'H32412')]


**When to use each:**
- `fetchall()`: Small result sets, when you need all data
- `fetchone()`: Processing results one by one, or just checking if results exist
- `fetchmany(size)`: Large datasets, memory management

In [41]:
import pandas as pd
df = pd.DataFrame(ans, columns=[col_name[0] for col_name in cursor.description])
df

,departement_id,departement_code,departement_nom,departement_nom_uppercase,departement_slug,departement_nom_soundex
0,1,01,Ain,AIN,ain,A500
1,2,02,Aisne,AISNE,aisne,A250
2,3,03,Allier,ALLIER,allier,A460
3,4,04,Alpes-de-Haute-Provence,ALPES-DE-HAUTE-PROVENCE,alpes-de-haute-provence,A412316152
4,5,05,Hautes-Alpes,HAUTES-ALPES,hautes-alpes,H32412


In [42]:
# Or even simpler with the built in function in pandas
df = pd.read_sql_query("SELECT * FROM departement LIMIT 5", conn)
df

,departement_id,departement_code,departement_nom,departement_nom_uppercase,departement_slug,departement_nom_soundex
0,1,01,Ain,AIN,ain,A500
1,2,02,Aisne,AISNE,aisne,A250
2,3,03,Allier,ALLIER,allier,A460
3,4,04,Alpes-de-Haute-Provence,ALPES-DE-HAUTE-PROVENCE,alpes-de-haute-provence,A412316152
4,5,05,Hautes-Alpes,HAUTES-ALPES,hautes-alpes,H32412
